# Goal

The goal of this demo is to demonstrate how the RandomNearIncumbentOptimizer works.

# Method

1. We create a 2-parameter single-objective synthetic function for the BayesianOptimizer to maximize.
2. We either use the pass-through model or train the initial model on a bunch of random parameters.
3. We plot:
    1. The original function
    2. The model's predictions
    3. The utility function values
    
4. We create the RandomNearIncumbentTracer and subscribe to all RandomNearIncumbentOptimizer events.
5. For each call to .suggest():
    1. We capture the initial incumbents and their utility values.
    2. For each call to _run_iteration():
        1. We capture the random neighbors and their utility values.
        2. We capture the new incumbents (and maybe even draw an arrow??)

6. We visualize all the data captured in 5. 

So for each scene we will need:
    1. Optionally the true objective function surface plot.
    2. Predicted value surface plot.
    3. Utility function surface plot.
    4. All past incumbents 3D scatter plot.
    5. All current incumbents 3D scatter plot.
    6. All current neighbors 3D scatter plot.

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import math

import mlos.global_values as global_values
from mlos.OptimizerEvaluationTools.ObjectiveFunctionFactory import ObjectiveFunctionFactory, objective_function_config_store
from mlos.OptimizerEvaluationTools.SyntheticFunctions.EnvelopedWaves import EnvelopedWaves, enveloped_waves_config_store
from mlos.Optimizers.BayesianOptimizerFactory import BayesianOptimizerFactory, bayesian_optimizer_config_store
from mlos.Optimizers.ExperimentDesigner.UtilityFunctions.ConfidenceBoundUtilityFunction import ConfidenceBoundUtilityFunction, confidence_bound_utility_function_config_store
from mlos.Optimizers.OptimizationProblem import OptimizationProblem, Objective
from mlos.Spaces import Point
from mlos.Tracer import Tracer

In [ ]:
# Create the objective function and the optimizer.
#
objective_function_config = Point(
    implementation=EnvelopedWaves.__name__,
    enveloped_waves_config=Point(
        num_params=2,
        num_periods=3,
        vertical_shift=0,
        phase_shift=0,
        period=2 * math.pi,
        envelope_type="linear",
        linear_envelope_config=Point(
            gradient=10
        )
    )
)
objective_function = ObjectiveFunctionFactory.create_objective_function(objective_function_config)
optimization_problem = OptimizationProblem(
    parameter_space=objective_function.parameter_space,
    objective_space=objective_function.output_space,
    objectives=[Objective(name='y', minimize=False)]
)
optimizer_config = bayesian_optimizer_config_store.get_config_by_name("default_with_random_near_incumbent_config")
#optimizer_config.experiment_designer_config.fraction_random_suggestions = 0.0 # For the purposes of this demo we only want to see guided suggestions.
optimizer_config.homogeneous_random_forest_regression_model_config.decision_tree_regression_model_config.n_new_samples_before_refit = 1 # We want the optimizer to be quite responsive.
optimizer_config.experiment_designer_config.utility_function_implementation = ConfidenceBoundUtilityFunction.__name__
optimizer_config.experiment_designer_config.confidence_bound_utility_function_config = confidence_bound_utility_function_config_store.default
optimizer_config.experiment_designer_config.random_near_incumbent_optimizer_config.initial_velocity = 0.1

optimizer_factory = BayesianOptimizerFactory()
optimizer = optimizer_factory.create_local_optimizer(optimizer_config=optimizer_config, optimization_problem=optimization_problem)

In [ ]:
optimizer_config

In [ ]:
import numpy as np
import pandas as pd

# First let's create a meshgrid of the parameters to later use to plot the objective function, the models prediction, and the utility function values.
#
resolution_px = 100
x_0_linspace = objective_function.parameter_space['x_0'].linspace(resolution_px)
x_1_linspace = objective_function.parameter_space['x_1'].linspace(resolution_px)
meshgrids = np.meshgrid(x_0_linspace, x_1_linspace)
reshaped_meshgrids = [meshgrid.reshape(-1) for meshgrid in meshgrids]
meshgrids_dict = {
    dim_name: meshgrid
    for dim_name, meshgrid
    in zip(objective_function.parameter_space.dimension_names, reshaped_meshgrids)
}
meshgrid_params_df = pd.DataFrame(meshgrids_dict)
meshgrid_features_df = optimization_problem.construct_feature_dataframe(parameters_df=meshgrid_params_df, product=True)

# True Objective Values

In [ ]:
import plotly.graph_objects as go

# Let's compute the true objectives and plot the true objective function surface.
#
true_objectives_df = objective_function.evaluate_dataframe(meshgrid_params_df)
reshaped_true_objectives = true_objectives_df['y'].to_numpy().reshape((resolution_px, resolution_px))
objective_function_surface = go.Surface(x=x_0_linspace, y=x_1_linspace, z=reshaped_true_objectives, opacity=0.3, name="True Objective Function Values")

In [ ]:
fig = go.Figure(
    data=go.Surface(x=x_0_linspace, y=x_1_linspace, z=reshaped_true_objectives, opacity=1, name="True Objective Function Values"),
    layout=go.Layout(
        title="Objective Function",
        hovermode="closest",
        width=1000,
        height=1000,
        showlegend=True
    )
)

fig.update_scenes(xaxis_title='x_0', yaxis_title='x_1', zaxis_title='y')

fig.show()

# Visualizing Optimization Process

In [ ]:
class FrameData:
    """Keeps track of all data for a given frame, and produces plotly traces for that frame.
    
    For each frame we need:
        1. Optionally the true objective function surface plot.
        2. Predicted value surface plot.
        3. Utility function surface plot.
        4. All past incumbents 3D scatter plot.
        5. All current incumbents 3D scatter plot.
        6. All current neighbors 3D scatter plot.
    """
    
    def __init__(
        self,
        predictions_df: pd.DataFrame = pd.DataFrame(columns=["x_0", "x_1", "predicted_value"], index=[]),
        utility_values_df: pd.DataFrame = pd.DataFrame(columns=["x_0", "x_1", "utility"], index=[]),
        all_observations_df: pd.DataFrame = pd.DataFrame(columns=["x_0", "x_1", "y"], index=[]),
        past_incumbents_df: pd.DataFrame = pd.DataFrame(columns=["x_0", "x_1", "utility"], index=[]),
        current_incumbents_df: pd.DataFrame = pd.DataFrame(columns=["x_0", "x_1", "utility"], index=[]),
        current_neighbors_df: pd.DataFrame = pd.DataFrame(columns=["x_0", "x_1", "utility"], index=[]),
        suggestion_df: pd.DataFrame = pd.DataFrame(columns=["x_0", "x_1", "utility"], index=[])
    ):
        self.predictions_df = predictions_df
        self.utility_values_df = utility_values_df
        self.all_observations_df = all_observations_df
        self.past_incumbents_df = past_incumbents_df
        self.current_incumbents_df = current_incumbents_df
        self.current_neighbors_df = current_neighbors_df
        self.suggestion_df = suggestion_df
        
        
        # Predicted value surface.
        #
        reshaped_predictions = self.predictions_df['predicted_value'].to_numpy().reshape((resolution_px, resolution_px))
        self.predicted_value_surface = go.Surface(x=x_0_linspace, y=x_1_linspace, z=reshaped_predictions, name="Predicted Values")
        
        # Utility values surface.
        #
        utility_values_df = pd.DataFrame(columns=['utility'], index=meshgrid_features_df.index)
        utility_values_df['utility'] = 0
        utility_values_df.loc[self.utility_values_df.index, 'utility'] = self.utility_values_df['utility']
        reshaped_utility_values = utility_values_df['utility'].to_numpy().reshape((resolution_px, resolution_px))
        self.utility_value_surface = go.Surface(x=x_0_linspace, y=x_1_linspace, z=reshaped_utility_values, name="Utility Function")
        
        
        self.all_observations_scatter_plot = go.Scatter3d(x=self.all_observations_df['x_0'], y=self.all_observations_df['x_1'], z=self.all_observations_df['y'], mode='markers', name="all_observations")
        self.past_incumbents_scatter_plot = go.Scatter3d(x=self.past_incumbents_df['x_0'], y=self.past_incumbents_df['x_1'], z=self.past_incumbents_df['utility'], mode='markers', name="past_incumbents")
        self.current_incumbents_scatter_plot = go.Scatter3d(x=self.current_incumbents_df['x_0'], y=self.current_incumbents_df['x_1'], z=self.current_incumbents_df['utility'], mode='markers', name="current_incumbents")
        self.current_neighbors_scatter_plot = go.Scatter3d(x=self.current_neighbors_df['x_0'], y=self.current_neighbors_df['x_1'], z=self.current_neighbors_df['utility'], mode='markers', name="current_neighbors")
        self.suggestion_scatter_plot = go.Scatter3d(x=self.suggestion_df['x_0'], y=self.suggestion_df['x_1'], z=self.suggestion_df['utility'], mode='markers', name="suggestion")
        
        
        
    def get_frame_data(self):        
        
        return [
            objective_function_surface,
            self.predicted_value_surface,
            self.utility_value_surface,
            self.all_observations_scatter_plot,
            self.past_incumbents_scatter_plot,
            self.current_incumbents_scatter_plot,
            self.current_neighbors_scatter_plot,
            self.suggestion_scatter_plot
        ]
    

In [ ]:
optimization_frames_data = []

In [ ]:
num_iterations = 500
for i in range(num_iterations):
    print(f"[{i}/{num_iterations}]")
    suggestion = optimizer.suggest()
    
    if (i % 5) == 0:
        # Let's capture a frame every 10 iterations.
        #
        predictions = optimizer.predict(meshgrid_params_df)
        predictions.add_invalid_rows_at_missing_indices(desired_index=meshgrid_params_df.index)
        predictions_df = predictions.get_dataframe()
        
        suggestion_features_df = optimization_problem.construct_feature_dataframe(parameters_df=suggestion.to_dataframe())
        suggestion_utility_value = optimizer.experiment_designer.utility_function(suggestion_features_df)
        suggestion_df = suggestion.to_dataframe()
        suggestion_df['utility'] = suggestion_utility_value['utility']
        
        params_df, objectives_df, context_df = optimizer.get_all_observations()
        all_observations_df = pd.concat([params_df, objectives_df], axis=1)
        
        frame_data = FrameData(predictions_df=predictions_df, suggestion_df=suggestion_df, all_observations_df=all_observations_df)
        optimization_frames_data.append(frame_data)
    
    suggestion_df = suggestion.to_dataframe()
    objective_df = objective_function.evaluate_dataframe(suggestion_df)
    optimizer.register(suggestion_df, objective_df)

In [ ]:
frames_data = [[objective_function_surface, frame.predicted_value_surface, frame.all_observations_scatter_plot] for frame in optimization_frames_data]

In [ ]:
first_frame = frames_data[0]

fig = go.Figure(
    data=first_frame,
    frames=[go.Frame(data=frame_data) for frame_data in frames_data],
    layout=go.Layout(
        title="Objective Function",
        hovermode="closest",
        updatemenus=[{
            "buttons":[
                {
                    "args": [None, {"fromcurrent": True}],
                    "label": "Play",
                    "method": "animate"
                },{
                    "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                      "mode": "immediate",
                                      "transition": {"duration": 0}}],
                    "label": "Pause",
                    "method": "animate"
                },{
                    "args": [None, {"fromcurrent": False}],
                    "label": "Restart",
                    "method": "animate"
                },
            ],
            "type":"buttons"
            
        }],
        width=1000,
        height=1000,
        showlegend=True
    )
)
fig.update_scenes(xaxis_title='x_0', yaxis_title='x_1', zaxis_title='y')
fig.show()

# Selecting the Next Configuration - Utility Function and its Optimizers

In [ ]:
# Set up the tracer to capture data during execution.
#
global_values.declare_singletons()
global_values.tracer = Tracer(actor_id="demo_notebook")

In [ ]:
class RandomNearIncumbentTracer:
    """Traces the execution of the RandomNearIncumbentOptimizer.
    The goal here is to capture data on every stage of the optimization:
        1. What were the original incumbents?
        2. What were the neighbors at a given iteration?
        3. What are the new incumbents after a given iteration
    This tracer subscribes to the events produced by the Tracer, and thus gets access to all the data we need.
    """

    def __init__(self):
        self.ordered_events = []

    def add_trace_event(self, name, phase, timestamp_ns, category, actor_id, thread_id, arguments):
        if name.startswith("RandomNearIncumbentOptimizer"):
            self.ordered_events.append(dict(
                name=name,
                phase=phase,
                arguments=arguments
            ))

    def clear_events(self):
        self.ordered_events = []

# Set up the random_near_incumbent_tracer to capture detailed data about it.
#
random_near_incumbent_tracer = RandomNearIncumbentTracer()
global_values.tracer.add_subscriber(event_callback=random_near_incumbent_tracer.add_trace_event)

In [ ]:
# The goal here is to capture raw data needed to prepare traces for each frame.
#
utility_function = optimizer.experiment_designer.utility_function
raw_frames_data = []
num_suggestions = 1
for i in range(num_suggestions):
    
    print(f"[{i}/{num_suggestions}]")
    
    utility_values_df=utility_function(meshgrid_features_df.copy(deep=True))
    
    predictions = optimizer.predict(meshgrid_params_df)
    predictions.add_invalid_rows_at_missing_indices(desired_index=meshgrid_params_df.index)
    predictions_df = predictions.get_dataframe()
    
    params_df, objectives_df, context_df = optimizer.get_all_observations()
    all_observations_df = pd.concat([params_df, objectives_df], axis=1)
    
    past_incumbents_df = pd.DataFrame(columns=["x_0", "x_1", "utility"])
    
    pre_suggestion_frame_data = FrameData(
        predictions_df=predictions_df,
        utility_values_df=utility_values_df,
        all_observations_df=all_observations_df
    )
    raw_frames_data.append(pre_suggestion_frame_data)
    
    random_near_incumbent_tracer.clear_events()
    suggested_params = optimizer.suggest()
    
    # Now we get to iterate over all events in the random_near_incumbent_tracer.ordered_events and produce FrameData objects as appropriate.
    #
    for event in random_near_incumbent_tracer.ordered_events:
        name = event["name"]
        phase = event["phase"]
        arguments = event["arguments"]
        
        if name == "RandomNearIncumbentOptimizer.suggest" and phase == "B":
            print(name, phase)
            # We are starting a new suggestion, let's clean up.
            #
            past_incumbents_df = pd.DataFrame(columns=["x_0", "x_1", "utility"])
        
        if name == "RandomNearIncumbentOptimizer._prepare_initial_params_df" and phase == "E":
            print(name, phase)
            # We have incumbents' params in the results object
            #
            current_incumbents_df = arguments["result"]
            features_df = optimization_problem.construct_feature_dataframe(parameters_df=current_incumbents_df.copy())
            utility_df = utility_function(features_df)
            current_incumbents_df['utility'] = utility_df['utility']
            
            past_incumbents_df = pd.concat([past_incumbents_df, current_incumbents_df], ignore_index=True)
            incumbents_frame_data = FrameData(
                predictions_df=predictions_df,
                utility_values_df=utility_values_df,
                all_observations_df=all_observations_df,
                current_incumbents_df=current_incumbents_df,
                past_incumbents_df=past_incumbents_df
            )
            raw_frames_data.append(incumbents_frame_data)
            
        if name == "RandomNearIncumbentOptimizer._prepare_random_neighbors" and phase == "E":
            print(name, phase)
            current_neighbors_df = arguments["result"][1]
            features_df = optimization_problem.construct_feature_dataframe(parameters_df=current_neighbors_df.copy())
            utility_df = utility_function(features_df)
            current_neighbors_df['utility'] = utility_df['utility']
            random_neighbors_frame_data = FrameData(
                predictions_df=predictions_df,
                utility_values_df=utility_values_df,
                all_observations_df=all_observations_df,
                current_incumbents_df=current_incumbents_df,
                current_neighbors_df=current_neighbors_df,
                past_incumbents_df=past_incumbents_df
            )
            raw_frames_data.append(random_neighbors_frame_data)
            
        if name == "RandomNearIncumbentOptimizer._run_iteration" and phase == "E":
            print(name, phase)
            # We have incumbents' params in the results object
            #
            current_incumbents_df = arguments["result"]
            features_df = optimization_problem.construct_feature_dataframe(parameters_df=current_incumbents_df.copy())
            utility_df = utility_function(features_df)
            current_incumbents_df['utility'] = utility_df['utility']
            
            past_incumbents_df = pd.concat([past_incumbents_df, current_incumbents_df], ignore_index=True)
            incumbents_frame_data = FrameData(
                predictions_df=predictions_df,
                utility_values_df=utility_values_df,
                all_observations_df=all_observations_df,
                current_incumbents_df=current_incumbents_df,
                current_neighbors_df=current_neighbors_df,
                past_incumbents_df=past_incumbents_df
            )
            raw_frames_data.append(incumbents_frame_data)
            
        
            
    
    # Compute suggestion_df
    #
    suggestion_features_df = optimization_problem.construct_feature_dataframe(parameters_df=suggested_params.to_dataframe())
    suggestion_utility_value = utility_function(suggestion_features_df)
    suggestion_df = suggested_params.to_dataframe()
    suggestion_df['utility'] = suggestion_utility_value['utility']
    
    suggestion_frame_data = FrameData(
        predictions_df=predictions_df,
        utility_values_df=utility_values_df,
        all_observations_df=all_observations_df,
        past_incumbents_df=past_incumbents_df,
        suggestion_df=suggestion_df
    )
    raw_frames_data.append(suggestion_frame_data)
    
    # Finally we can register an observation with the optimizer.
    #
    suggested_params_df = suggested_params.to_dataframe()
    objectives_df = objective_function.evaluate_dataframe(suggested_params_df)
    optimizer.register(suggested_params_df, objectives_df)

In [ ]:
past_incumbents_df

In [ ]:
first_frame = raw_frames_data[0].get_frame_data()

fig = go.Figure(
    data=first_frame,
    frames=[go.Frame(data=frame_data.get_frame_data()) for frame_data in raw_frames_data],
    layout=go.Layout(
        title="Objective Function",
        hovermode="closest",
        updatemenus=[{
            "buttons":[
                {
                    "args": [None, {"fromcurrent": True}],
                    "label": "Play",
                    "method": "animate"
                },{
                    "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                      "mode": "immediate",
                                      "transition": {"duration": 0}}],
                    "label": "Pause",
                    "method": "animate"
                },{
                    "args": [None, {"fromcurrent": False}],
                    "label": "Restart",
                    "method": "animate"
                },
            ],
            "type":"buttons"
            
        }],
        width=1000,
        height=1000,
        showlegend=True
    )
)
fig.update_scenes(xaxis_title='x_0', yaxis_title='x_1', zaxis_title='y')
fig.show()